In [13]:
# Package installs
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install scikit-learn
%pip install cartopy
%pip install keras
%pip install keras-tuner
# you may have to install in console using pip install tensorflow --user or pip install tensorflow --upgrade
%pip install tensorflow
%pip install basketball-refrence-scraper

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Brian\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Brian\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Brian\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Brian\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement basketball-refrence-scraper (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Brian\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for basketball-refrence-scraper


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# Modeling tools
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
import keras_tuner as kt
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import OutputType
import ace_tools as tools

ModuleNotFoundError: No module named 'ace_tools'

In [9]:
# Get the current working directory
current_directory = os.getcwd()

# Define the path to the dataset
dataset_path = os.path.join(current_directory, 'player.csv')

# Load the dataset
df = pd.read_csv(dataset_path, delimiter=',', encoding='ISO-8859-1')

# Check the first few rows
df.head(10)

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,0
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,0
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,0
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,0
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,0
5,949,Shareef Abdur-Rahim,Shareef,Abdur-Rahim,0
6,76005,Tom Abernethy,Tom,Abernethy,0
7,76006,Forest Able,Forest,Able,0
8,76007,John Abramovic,John,Abramovic,0
9,203518,Alex Abrines,Alex,Abrines,0


In [4]:
print(df.isnull())

      person_id  first_name  last_name  display_first_last  \
0         False       False      False               False   
1         False       False      False               False   
2         False       False      False               False   
3         False       False      False               False   
4         False       False      False               False   
...         ...         ...        ...                 ...   
4166      False       False      False               False   
4167      False       False      False               False   
4168      False       False      False               False   
4169      False       False      False               False   
4170      False       False      False               False   

      display_last_comma_first  display_fi_last  player_slug  birthdate  \
0                        False            False        False      False   
1                        False            False        False      False   
2                        False

In [5]:
df.drop(columns=['birthdate' , 'display_first_last' ,  ])

,person_id,first_name,last_name,display_last_comma_first,display_fi_last,player_slug,school,country,last_affiliation,height,...,playercode,from_year,to_year,dleague_flag,nba_flag,games_played_flag,draft_year,draft_round,draft_number,greatest_75_flag
0,76001,Alaa,Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,Duke,USA,Duke/USA,6-10,...,HISTADD_alaa_abdelnaby,1990.0,1994.0,N,Y,Y,1990,1,25,N
1,76003,Kareem,Abdul-Jabbar,"Abdul-Jabbar, Kareem",K. Abdul-Jabbar,kareem-abdul-jabbar,UCLA,USA,UCLA/USA,7-2,...,HISTADD_kareem_abdul-jabbar,1969.0,1988.0,N,Y,Y,1969,1,1,Y
2,1505,Tariq,Abdul-Wahad,"Abdul-Wahad, Tariq",T. Abdul-Wahad,tariq-abdul-wahad,San Jose State,France,San Jose State/France,6-6,...,tariq_abdul-wahad,1997.0,2003.0,N,Y,Y,1997,1,11,N
3,949,Shareef,Abdur-Rahim,"Abdur-Rahim, Shareef",S. Abdur-Rahim,shareef-abdur-rahim,California,USA,California/USA,6-9,...,shareef_abdur-rahim,1996.0,2007.0,N,Y,Y,1996,1,3,N
4,76005,Tom,Abernethy,"Abernethy, Tom",T. Abernethy,tom-abernethy,Indiana,USA,Indiana/USA,6-7,...,HISTADD_tom_abernethy,1976.0,1980.0,N,Y,Y,1976,3,43,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,1627835,Paul,Zipser,"Zipser, Paul",P. Zipser,paul-zipser,Bayern Munich,Germany,Bayern Munich/Germany,6-8,...,paul_zipser,2016.0,2017.0,Y,Y,Y,2016,2,48,N
4167,1627790,Ante,Zizic,"Zizic, Ante",A. Zizic,ante-zizic,Darussafaka,Croatia,Darussafaka/Croatia,6-10,...,ante_zizic,2017.0,2019.0,Y,Y,Y,2016,1,23,N
4168,78647,Jim,Zoet,"Zoet, Jim",J. Zoet,jim-zoet,Kent State,USA,Kent State/USA,7-1,...,HISTADD_jim_zoet,1982.0,1982.0,N,Y,Y,Undrafted,Undrafted,Undrafted,N
4169,1627826,Ivica,Zubac,"Zubac, Ivica",I. Zubac,ivica-zubac,Mega Basket,Croatia,Mega Basket/Croatia,7-0,...,ivica_zubac,2016.0,2023.0,Y,Y,Y,2016,2,32,N


In [ ]:
df = df[df["is_active"] == 1]

In [12]:
df.to_csv("ActivePlayers.csv", index=False)

In [17]:
player_stats = client.players_season_totals(season_end_year=2025)
print(player_stats)  # List of all players' season stats

[{'slug': 'gilgesh01', 'name': 'Shai Gilgeous-Alexander', 'positions': [<Position.POINT_GUARD: 'POINT GUARD'>], 'age': 26, 'team': <Team.OKLAHOMA_CITY_THUNDER: 'OKLAHOMA CITY THUNDER'>, 'games_played': 49, 'games_started': 49, 'minutes_played': 1668, 'made_field_goals': 561, 'attempted_field_goals': 1051, 'made_three_point_field_goals': 97, 'attempted_three_point_field_goals': 279, 'made_free_throws': 382, 'attempted_free_throws': 423, 'offensive_rebounds': 45, 'defensive_rebounds': 212, 'assists': 294, 'steals': 95, 'blocks': 50, 'turnovers': 131, 'personal_fouls': 107, 'points': 1601}, {'slug': 'edwaran01', 'name': 'Anthony Edwards', 'positions': [<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>], 'age': 23, 'team': <Team.MINNESOTA_TIMBERWOLVES: 'MINNESOTA TIMBERWOLVES'>, 'games_played': 51, 'games_started': 51, 'minutes_played': 1873, 'made_field_goals': 466, 'attempted_field_goals': 1045, 'made_three_point_field_goals': 212, 'attempted_three_point_field_goals': 504, 'made_free_throws': 

In [18]:
df1 = pd.DataFrame(player_stats)
df1.to_csv("nba_player_stats.csv", index=False)

In [32]:
df2 = pd.DataFrame(player_stats)

# Calculate per-game averages
df2["points_per_game"] = df2["points"] / df2["games_played"]
df2["assists_per_game"] = df2["assists"] / df2["games_played"]
df2["total_rebounds"] = df2["offensive_rebounds"] + df2["defensive_rebounds"]
df2["rebounds_per_game"] = df2["total_rebounds"] / df2["games_played"]
df2["steals_per_game"] = df2["steals"] / df2["games_played"]
df2["blocks_per_game"] = df2["blocks"] / df2["games_played"]
df2["turnovers_per_game"] = df2["turnovers"] / df2["games_played"]

# Display only relevant columns
df2 = df2[["name", "points_per_game", "assists_per_game", "rebounds_per_game","blocks_per_game" , "turnovers_per_game"]]

In [33]:
df2.to_csv("average_stats.csv", index=False)